The elo system for two players A and B with $E$ being the expected score (chance of winning in percent) and $R$ being the rating of the players says:
$$E_{A}={\frac {1}{1+10^{(R_{B}-R_{A})/400}}}$$
or conversely
$$E_{B}={\frac {1}{1+10^{(R_{A}-R_{B})/400}}}$$

Further, the adjustment after a match, with $R'$ being the updated rating, $S$ being the outcome, and $K$ being a scaling factor:
$$R_{A}^{\prime }=R_{A}+K(S_{A}-E_{A})$$
This scaling factor of $K$ represents the maximum the score can be adjusted by, and is apparently often adjusted depending on the player's current score. An example $K$ equation from the USCF, with $N_e$ being the effective number of games a rating is based on and $m$ being the number of games in a tournament:
$$K=800/(N_{e}+m)$$
Other implementations seem to be without formulas, e.g.:
$$K = \begin{cases}40 & \textrm{less than 30 games played as long as} R < 2300 \\ 20 & R \leq 2300 \\ 10 & R > 2300\end{cases}$$

In [174]:
def E(R):
    return (1 + 10 ** ((R[1] - R[0]) / 400)) ** -1

def R_1v1(R, S, K=None, N=1):
    if K is None:
        K = Kc(N, R[0])
    return R[0] + K * (S - E(R))

def new_R(R, S, E, K=None, N=None):
    if K is None:
        K = Kc(N, R)
    return R + K * (S - E)

def Kc(N, R):
    if N < 30 and R < 2300:
        return 40
    elif R <= 2300:
        return 20
    else: # R > 2300
        return 10

An example wrapper:

In [297]:
ratings_db = {"Dellpit": [1000, 1], "MrEirox": [1000, 1], "Tha Fazz": [1250, 1], "Jelko": [1000, 1], "Lime": [1000, 1], "DurZa": [1000, 1]}

In [298]:
def team_ratings(team_1, team_2, outcome):
    
    # team sizes
    l = len(team_1)
    
    # make sure team_1 and team_2 have same length
    if l != len(team_2):
        raise ValueError("team_1 and team_2 must have the same length!")
    
    # set S value according to outcome
    if outcome == 1:
        S = [1, 0]
    elif outcome == 2:
        S = [0, 1]
    elif outcome == 0:
        S = [.5, .5]
    else:
        raise ValueError("outcome must be 1 for team_1, 2 for team_2, or 0 for a tie!")
    
    # read out player data from db
    t1 = []
    t2 = []
    for i in range(l):
        t1.append(ratings_db[team_1[i]])
        t2.append(ratings_db[team_2[i]])
    
    # calculate total rating for each team 
    R_old_1 = 0
    R_old_2 = 0
    
    for i in range(l):
        R_old_1 = R_old_1 + (t1[i])[0]
        R_old_2 = R_old_2 + (t2[i])[0]
    
    # calculate expected outcome for each team
    E_1 = E([R_old_1 / l, R_old_2 / l])
    E_2 = 1 - E_1
    
    # update values in database
    for i in range(l):
        ratings_db[team_1[i]] = [int(new_R(R=(t1[i])[0], S=S[0], E=E_1, N=(t1[i])[1])), (t1[i])[1] + 1]
        ratings_db[team_2[i]] = [int(new_R(R=(t2[i])[0], S=S[1], E=E_2, N=(t2[i])[1])), (t2[i])[1] + 1]
        
    print("Ratings successfully adjusted.")
    
    return

In [299]:
team_ratings(team_1=["Dellpit", "MrEirox", "Lime"], team_2=["Tha Fazz", "Jelko", "DurZa"], outcome=1)

Ratings successfully adjusted.


In [300]:
print(ratings_db)

{'Dellpit': [1024, 2], 'MrEirox': [1024, 2], 'Tha Fazz': [1225, 2], 'Jelko': [975, 2], 'Lime': [1024, 2], 'DurZa': [975, 2]}


In [274]:
print(E([2000, 2500])) # 2 * 1000 vs 2 * 1250
print(E([2000 / 2, 2500 / 2]))
print(E([3000, 3750])) # 3 * 1000 vs 3 * 1350
print(E([3000 / 3, 3750 / 3]))

0.05324021520202244
0.19168232745054142
0.0131597265477055
0.19168232745054142
